# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("event")
except Exception as e:
    print(e)

### Before getting into quries we first need to see the data and how it is structured and also create our pandas dataframe to work with

In [8]:
df = pd.read_csv('event_datafile_new.csv')

In [9]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [11]:
df.describe()

,itemInSession,length,sessionId,userId
count,6820.000000,6820.000000,6820.000000,6820.000000
mean,22.761144,247.032221,599.181818,54.681232
std,23.444636,102.975921,284.953333,28.162734
min,0.000000,15.855870,3.000000,2.000000
25%,4.000000,197.321998,374.000000,29.000000
50%,15.000000,232.972605,605.000000,49.000000
75%,35.000000,274.121992,834.000000,80.000000
max,126.000000,2594.873020,1114.000000,101.000000


In [12]:
df.select_dtypes(include='object').describe()

,artist,firstName,gender,lastName,level,location,song
count,6820,6820,6820,6820,6820,6820,6820
unique,3148,84,2,86,2,63,5190
top,Coldplay,Chloe,F,Cuevas,paid,"San Francisco-Oakland-Hayward, CA",You're The One
freq,58,703,4887,689,5591,691,37


In [13]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [14]:
# Testing to see if our composite key is unique or not
cols = ['sessionId', 'itemInSession']
df[cols].duplicated().sum()

0

In [15]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
    CREATE TABLE IF NOT EXISTS sessions
    (sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY (sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, length from sessions WHERE  sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [18]:
# Testing to see if our composite key is unique or not
cols = ['sessionId', 'userId', 'itemInSession']
df[cols].duplicated().sum()

0

In [19]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
    CREATE TABLE IF NOT EXISTS events
    (userId int,
     sessionId int,
     itemInSession int,
     artist text, 
     song text, 
     userName text, 
     primary key (userId, sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                     

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events (userId, sessionId, itemInSession, artist, song, userName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + ' ' + line[4]))

In [22]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, userName from events WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.username)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [23]:
# Testing to see if our composite key is unique or not
cols = ['song', 'firstName', 'lastName']
df[cols].duplicated().sum()

202

#### As we can find here we have douplicate values but it's okay as we only need to see who listned to a givin song

In [24]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    CREATE TABLE IF NOT EXISTS users
    (song text, 
     userName text, 
     primary key (song, userName))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                     

In [25]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users (song, userName)"
        query = query + " VALUES (%s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

In [26]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select userName from users WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.username)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [27]:
## TO-DO: Drop the table before closing out the sessions
session.execute("drop table sessions")
session.execute("drop table events")
session.execute("drop table users")

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()